## 卷积神经网络简介
卷积神经网络最初是为解决图像识别等问题设计的，当然其现在的应用不仅限于图像和视频，也可用于时间序列信号，比如音频信号、文本数据等。
 
在早期的图像识别研究中，最大的挑战是如何组织特征，因为图像数据不像其他类型的数据那样可以通过人工理解来提取特征。
 
在股票预测等模型中，我们可以从原始数据中提取过往的交易价格波动、市盈率、市净率、盈利增长等金融因子，这既是特征工程。
 
但是在图像中，我们很难根据人为理解提取出有效而丰富的特征。
 
在深度学习出现之前，我们必须辅助SIFT、HoG等算法提取具有良好区分性的特征，再集合SVM等机器学习算法进行图像识别。
 
SIFT对一定程度内的缩放、平移、旋转、视角改变、亮度调整等畸变，都具有不变性，是当时最重要的图像特征提取方法之一。
 
在之前只能依靠SIFT等特征提取算法才能勉强进行可靠的图像识别。
 
CNN可以直接使用图像的原始像素作为输入，而不必先使用SIFT等算法提取特征，减轻了使用传统算法如SVM时必需要做得大量重复、繁琐的数据预处理工作。
 
CNN的最大特点在于卷积的权值共享结构，可以大幅减少神经网络的参数量，防止过拟合的同时又降低了神经网络的复杂度。
 
一般的卷积神经网络由多个卷积层构成，每个卷积层中通常会进行如下几个操作:
 1. 图像通过多个不同的卷积核的滤波，并加偏置(bias),提取出局部特征，每一个卷积核会映射出一个新的2D图像
 2. 将前面卷积核的滤波输出结果，进行非线性的激活函数处理。目前最常见的是使用ReLU函数，而以前sigmoid函数用得比较多
 3. 对激活函数的结果再进行池化操作(即降采样，比如将2x2的图片降为1x1的图片)，目前一般是使用最大池化，保留最显著的特征，并提升模型的畸变容忍能力
 <br>
 
这几个步骤就构成了最常见的卷积层，当然也可以再加上一个LRN（局部响应归一化层）层，目前非常流行的Trick还有Batch Normalization等。
 
一个卷积层中可以有多个不同的卷积核，而每一个卷积核都对应一个滤波后映射出的新图像，同一个新图像中每一个图像都来自完全相同的卷积核，这就是卷积核的全职共享。
 
这一小块区域内的像素是相互关联的，每一个神经元不需要接收全部像素点的信息，只需要接收局部的像素点作为输入，而后将所有这些神经元收到的局部信息综合起来就可以得到全局的信息。
这样就可以将之前的全连接的模式修改为局部连接，之前隐含层的每一个隐含节点都和全部像素相连，现在我们只需要将每一个隐含节点连接到局部的像素节点。
 
通过局部连接的方法，将连接数从1万亿降低到1亿，但仍然偏多，需要继续降低参数量。现在隐含层每一个节点都与10x10的像素相连，也就是每一个隐含节点都拥有100个参数。假设我们的局部连接方式是卷积操作，即默认每一个**隐藏节点的参数都完全一样**，那我们的参数不再是1亿，而是100。**参数量只跟卷积核的大小有关，这也就是所谓的权值共享**。
 
卷积神经网络的要点就是局部连接、权值共享和池化层中的降采样。其中，局部连接和权值共享降低了参数数量，使训练复杂度降低，并减轻了过拟合。
 
同时权值共享还赋予了卷积神经网络对平移的容忍性，而池化层降采样则进一步降低了输出餐数量，并赋予模型对轻度型变得容忍性，提高了模型的泛化能力。
 
LeNet5当时的特性有如下几点：
1. 每个卷积层包含三个部分：卷积、池化和非线性激活函数
2. 使用卷积提取空间特征
3. 降采样(Subsample)的平均池化层(Average Pooling)
4. 双曲正切(Tanh)或S型(Sigmoid)的激活函数
5. MLP作为最后的分类器
6. 层与层之间的稀疏连接减少计算复杂度
  

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
#创建默认的InteractiveSession
sess = tf.InteractiveSession()

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [0]:
def weight_variable(shape):
  #用于重复初始化权重
  #使用截断的正态分布噪声来打破完全对称，标准差设为0.1
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)
  
def bias_variable(shape):
  #用于重复初始化偏置
  #使用ReLU，也需要给偏置增加一些小的正值0.1来避免死亡节点
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

卷积层、池化层也是接下来要重复使用的，因此也为他们分别定义创建函数。

In [0]:
def conv2d(x,W):
  '''
  tf.nn.conv2d是tf中的2维卷积函数。
  x：输入
  W：卷积的参数，如[5,5,1,32],前两个表示卷积核的尺寸，第三个表示有多少个channel，因为是灰度，所以为1，如果是RGB的为3，最后一个表示卷积核的数量。
  strides：代表卷积模板移动的步长
  padding:代表边界的处理方式。SAME表示给边界加上Padding让卷积的输出和输入保持同样的尺寸。
  '''
  return tf.nn.conv2d(x,W,strides=[1,1,1,1], padding='SAME')

def max_pool_2x2(x):
  '''
  strides代表卷积模块移动的步长，都是1代表会不遗漏的划过的图片的每一个点
  padding代表卷积模板移动的步长，这里的SAME代表给边界加上padding让卷积的输出和输入保持同样的尺寸
  tf.nn.max_pool即将一个2x2的像素降为1x1的像素。最大池化会保留原始像素块中灰度值最高的那一个像素，即保留最显著的特征。
  '''
  return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1],padding='SAME')

In [0]:
#定义输入的placeholder
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])

#因为卷积神经网络会利用空间结构信息，因此需要将1D的输入向量转为2D的图片结构，即从1x784的形式转为原始的28x28的结构。[-1,28,28,1]，-1表示样本数量不固定，1代表颜色通道数量
x_image = tf.reshape(x, [-1,28,28,1])

In [0]:
#定义第一个卷积层。使用前面写好的函数进行参数初始化，包括weights和bias
W_conv1 = weight_variable([5,5,1,32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1)+b_conv1)
h_pool1 = max_pool_2x2(h_conv1)#对卷积的输出结果进行池化操作

In [0]:
#定义第二个卷积层，唯一不同的是，这一层的卷积核数量为64
W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2)+b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [0]:
#最后一个全连接层
W_fc1 = weight_variable([7*7*64,1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)

In [0]:
#使用一个dropout层。用于减轻过拟合。在训练时，我们通过随机丢弃一部分节点的数据来减轻过拟合，预测时则保留全部数据来追求最好的预测性能
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [0]:
#softmax层，得到最后的概率输出
W_fc2 = weight_variable([1024,10])
b_fc2 = bias_variable([10])
y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2)+b_fc2)

In [0]:
#定义损失函数为cross entropy和之前一样，但是优化器使用Adam，并使用一个比较小的学习速率
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_*tf.log(y_conv),reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [0]:
#再继续定义评测准确率的操作
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [0]:
#开始训练过程。首先初始化所有参数，设置训练时dropout的keep_prob为0.5
tf.global_variables_initializer().run()
for i in range(20000):
  batch = mnist.train.next_batch(50)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={x:batch[0],y_:batch[1],keep_prob:1.0})
    
  train_step.run(feed_dict={x:batch[0], y_:batch[1], keep_prob:0.5})

In [0]:
print('test accuracy %g' % accuracy.eval(feed_dict={
    x:mnist.test.images,y_:mnist.test.labels,keep_prob:1.0
}))